This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [2]:
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [4]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [10]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

# Specify the url and parameters
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json'
params = {'api_key': API_KEY}

# Create the response object
r = requests.get(url, params=params)

myDict = dict(r.json())

In [24]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(myDict.keys())
print(myDict['dataset_data'].keys())

for item in myDict['dataset_data'].keys():
    print(item + ':', type(myDict['dataset_data'][item]))

print()
for item in (myDict['dataset_data'].keys() - ['data']):
    print(item + ':', myDict['dataset_data'][item])

dict_keys(['dataset_data'])
dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])
limit: <class 'NoneType'>
transform: <class 'NoneType'>
column_index: <class 'NoneType'>
column_names: <class 'list'>
start_date: <class 'str'>
end_date: <class 'str'>
frequency: <class 'str'>
data: <class 'list'>
collapse: <class 'NoneType'>
order: <class 'NoneType'>

start_date: 2000-06-07
limit: None
column_names: ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
end_date: 2020-12-01
column_index: None
frequency: daily
collapse: None
order: None
transform: None


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#### Collect 2017 data and put into a dictionary

In [25]:
# Specify the url and parameters
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json'
params = {'api_key': API_KEY, 'start_date': '2017-01-01', 'end_date': '2017-12-31'}

# Create the response object
r = requests.get(url, params=params)

myDict = dict(r.json())

for item in (myDict['dataset_data'].keys() - ['data']):
    print(item + ':', myDict['dataset_data'][item])

start_date: 2017-01-01
limit: None
column_names: ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
end_date: 2017-12-31
column_index: None
frequency: daily
collapse: None
order: None
transform: None


#### Highest and lowest opening prices for this period of time

In [42]:
myData = myDict['dataset_data']['data']
inval = []
for item in myData:
    if not item[1]: # Noticed some of the days didn't have an opening price, so I took them out for calculating max and min
        inval.append(item)
        
cleanOpening = [item for item in myData if item not in inval]

In [43]:
max_day = max(cleanOpening, key=lambda a: a[1])
min_day = min(cleanOpening, key=lambda a: a[1])

print("Highest opening:", max_day)
print("Lowest opening:", min_day)

Highest opening: ['2017-12-14', 53.11, 53.54, 52.15, 52.67, None, 132981.0, 7016953.0, None, None, None]
Lowest opening: ['2017-01-24', 34.0, 34.35, 33.85, 34.22, None, 48797.0, 1666086.0, None, None, None]


#### Largest change in one day (based on High and Low price)

In [47]:
max_change = max(myData, key=lambda a: abs(a[2] - a[3]))

print("Largest change:", max_change)

Largest change: ['2017-05-11', 43.4, 46.06, 43.25, 45.0, None, 189125.0, 8496322.0, None, None, None]


#### Largest change between two days

In [52]:
day_changes = [abs(item[4] - item_next[4]) for item, item_next in zip(myData[:-1], myData[1:])]
max_delta_idx = day_changes.index(max(day_changes)) + 1

print("Largest change between 2 days:", myData[max_delta_idx - 1], myData[max_delta_idx])

Largest change between 2 days: ['2017-08-09', 43.5, 43.5, 41.64, 41.81, None, 355857.0, 15003956.0, None, None, None] ['2017-08-08', 44.9, 45.09, 44.15, 44.37, None, 156168.0, 6941408.0, None, None, None]


#### Average daily trading volume during this year

In [57]:
import statistics

avg_vol = statistics.mean([item[6] for item in myData])

print("Average daily trading volume:", avg_vol)

Average daily trading volume: 89124.33725490196


#### Finding the median trading volume

In [59]:
idx_trading_vol = [(idx,item) for idx, item in enumerate(myData)]
sorted_trading_vol = sorted(idx_trading_vol, key=lambda a: a[1][6])

median_idx = sorted_trading_vol[len(sorted_trading_vol)//2][0]
median = myData[median_idx]

print(median)

['2017-11-13', 48.38, 48.61, 47.76, 48.34, None, 76286.0, 3681337.0, None, None, None]
